In [1]:
!pip install adtk
!pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 2.9 MB/s eta 0:00:0000:0100:01


In [1]:
import pandas as pd
import numpy as np
from pandas.plotting import scatter_matrix
#import time

from adtk.data import validate_series
from adtk.visualization import plot
from adtk.detector import ThresholdAD
from adtk.detector import OutlierDetector

from sklearn.neighbors import LocalOutlierFactor
#from sklearn.metrics import r2_score

from scipy.stats import variation

import seaborn as sns

import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_pacf,plot_acf
from statsmodels.stats.outliers_influence import variance_inflation_factor

#from dateutil.parser import parse

#import itertools
from itertools import compress, product

#import pmdarima as pm
from pmdarima import auto_arima

import warnings
warnings.filterwarnings("ignore") # specify to ignore warning messages

# функция графика сезонности
def sesonal(data, s):
    plt.figure(figsize=(19,8), dpi= 80)
    for i, y in enumerate(data.index.year.unique()):
        plt.plot(list(range(1,len(data[data.index.year==y])+1)), data[data.index.year==y][data.columns[0]].values, label=y)
    plt.title("Сезонность по периодам")
    plt.legend(loc="best")
    plt.show()
    
def metrics(real, forecast):
    
    if type(real)==pd.core.frame.DataFrame:
        real=real[real.columns[0]].values
    
    print("Тест на стационарность:")
    dftest = adfuller(real-forecast, autolag='AIC')
    print("\tT-статистика = {:.3f}".format(dftest[0]))
    print("\tP-значение = {:.3f}".format(dftest[1]))
    print("Критические значения :")
    for k, v in dftest[4].items():
        print("\t{}: {} - Данные {} стационарны с вероятностью {}% процентов".format(k, v, "не" if v<dftest[0] else "", 100-int(k[:-1])))
    
    #real=np.array(real[real.columns[0]].values)
    forecast=np.array(forecast)
    print('MAD:', round(abs(real-forecast).mean(),4))
    print('MSE:', round(((real-forecast)**2).mean(),4))
    print('MAPE:', round((abs(real-forecast)/real).mean(),4))
    print('MPE:', round(((real-forecast)/real).mean(),4))
    print('Стандартная ошибка:', round(((real-forecast)**2).mean()**0.5,4)) 
    

def metrics_short(real, forecast):
    real=np.array(real[real.columns[0]].values)
    forecast=np.array(forecast)
    print('MAD:', round(abs(real-forecast).mean(),4))
    print('MSE:', round(((real-forecast)**2).mean(),4))
    print('MAPE:', round((abs(real-forecast)/real).mean(),4))
    print('MPE:', round(((real-forecast)/real).mean(),4))
    print('Стандартная ошибка:', round(((real-forecast)**2).mean()**0.5,4)) 
    
def h_map(data, level):
    corr = data.corr()
    plt.figure(figsize=(14, 14))
    sns.heatmap(corr[(corr >= level) | (corr <= -level)],
            cmap="RdBu_r", vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True)
    plt.show()
    
#небольшая функция для построения набора комбинаций переменных
def combinations(items):
    return list( set(compress(items,mask)) for mask in product(*[[0,1]]*len(items)) )

def get_factors(data, Y, columns):

    # колонки, которые показали свою значимость в процессе отбора критериев
    # переменная spisCol хранит варианты комбинаций все переменных
    spisCol=combinations(columns)

    print('Количество комбинаций ', len(spisCol))
    
    #добавим константу в набор данных, нужна для рассчета регрессии
    data=sm.add_constant(data)

    #сохраним в этом списке данные лучших моделей
    arr_res=[]

    #пробежимся циклом по всем вариантам комбинаций
    for c in spisCol:
        perem=list(c)
        flag=True
    
        if len(perem)==0: continue
        
        if not('const' in c):
            perem.append('const')
        
        # если больше одного клитерия, рассчитаем VIF    
        if len(perem)>1:
            vif = [variance_inflation_factor(data[perem].values, i) for i in range(data[perem].shape[1])]
        else:
            vif=[]
    
        #проверим список VIF, если хоть одна переменная больше 1000 (очень большое значение, на самом деле),
        #то в модели присутсвует мультиколлинераность
        for vv in vif:
            if vv>1000: 
                flag=False
        
        #посчитаем саму модель
        reg = sm.OLS(Y, data[perem])
        res=reg.fit()

        #отбросим нулевую гипотезу для всех регрессоров конкретной модели
        for val in res.tvalues:
            if val<2 and val>-2:
                flag=False
                break
        for val in res.pvalues:
            if val>0.05:
                flag=False
                break
        #если нулевую гипотезу отбросили и VIF в норме, сохраним результаты
        if flag:
            re=np.array(res.fittedvalues.copy())
            MSE=((np.array(Y)-re)**2).sum()/len(re)
            
            MAPE=(abs((np.array(Y)-re)/np.array(Y))).sum()/len(re)
        
            arr_res.append([round(MSE,4), res.rsquared, perem])

    #отсортируем и выведем результаты
    arr_res.sort()
    df_model=pd.DataFrame(arr_res, columns=['MSE', 'r2', 'Переменные'])
    print('Результаты перебора в порядке возрастания MSE:')
    print(df_model)
    return df_model

In [9]:
# import read data from xlsx in pandas,
# rotate and use first row as header
def read_xlsx(path):
    df = pd.read_excel(path, engine='openpyxl')
    df = df.transpose()
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    # drop index
    df = df.reset_index(drop=True)

    return df

# read data
df_y_train = read_xlsx(f"../data/raw/Y_train.xlsx")
df_y_train.head()

"ИПЦ, мом",Период,Целевой показатель
0,2020-06-01 00:00:00,0.28
1,2020-07-01 00:00:00,-0.065
2,2020-08-01 00:00:00,-0.005
3,2020-09-01 00:00:00,0.315
4,2020-10-01 00:00:00,0


In [10]:
# Convert Период to datetime
df_y_train['Период'] = pd.to_datetime(df_y_train['Период'], format='%Y-%m-%d')


In [11]:
df_y_train.set_index('Период')
df_y_train.head()

"ИПЦ, мом",Период,Целевой показатель
0,2020-06-01,0.28
1,2020-07-01,-0.065
2,2020-08-01,-0.005
3,2020-09-01,0.315
4,2020-10-01,0


In [13]:
df_y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Период              24 non-null     datetime64[ns]
 1   Целевой показатель  24 non-null     object        
dtypes: datetime64[ns](1), object(1)
memory usage: 512.0+ bytes


In [12]:
start_date = '2020-01-01'
end_date = '2022-01-01'
train = df_y_train[start_date:end_date]

TypeError: cannot do slice indexing on RangeIndex with these indexers [2020-01-01] of type str

In [ ]:
model = auto_arima(train, seasonal=True, m=12, trace=True, suppress_warnings=True, error_action='ignore', stepwise=True)
model

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train,
                                order=(1, 0, 0),
                                seasonal_order=(2, 0, 0, 12))

results = mod.fit()

print(results.summary().tables[1])

In [ ]:
results.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
predict=results.get_prediction()
predict.predicted_mean[:10]

In [ ]:
predict=results.get_prediction(start='2014-02-01')
metrics(train['2014-02-01':], predict.predicted_mean)

In [ ]:
predict=results.get_prediction(start='2019', end='2021')

In [ ]:
ax = df_y_train.plot(figsize=(15,6), color='black', title="Прогноз методом SARIMA" )
results.fittedvalues.plot(ax=ax, style='--', color='red')
predict.predicted_mean.plot(ax=ax, style='--', color='green')
plt.show()